In [84]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from os.path import join


from keras.preprocessing import image
from keras.applications import xception
from keras.layers import Input, Lambda, Dense
from keras.models import Model, Sequential

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score
from sklearn.preprocessing import LabelEncoder

In [42]:
competition_name = 'dog-breed-identification'
data_dir = '/opt/notebooks/data/' + competition_name + '/unzipped/'

In [58]:
num_classes = 10

In [60]:
labels = pd.read_csv(data_dir+'labels.csv')

selected_breed_list = labels.breed.value_counts().index.values[:num_classes]
selected_breed_list = list(selected_breed_list)
labels = labels[labels.breed.isin(selected_breed_list)]

In [61]:
y_train = pd.get_dummies(labels.breed).values

In [62]:
le = LabelEncoder()
y_train_encode = le.fit_transform(labels.breed.values)

In [63]:
rnd = np.random.random(len(labels))

train_idx = rnd < 0.8
valid_idx = rnd >= 0.8
ytr = y_train[train_idx]
yv = y_train[valid_idx]

### Use output of bottleneck xception from preprocessed input

In [64]:
x_train = np.empty((len(labels), 299, 299, 3), dtype='float32')
def read_img(img_id, train_or_test, size):
    img = image.load_img(join(data_dir, train_or_test, '%s.jpg' % img_id), target_size=size)
    img = image.img_to_array(img)
    return img

for i, img_id in enumerate(labels['id']):
    img = read_img(img_id, 'train', (299, 299))
    x = xception.preprocess_input(img)
    x_train[i] = x

Xtr = x_train[train_idx]
Xv = x_train[valid_idx]

In [67]:
xception_bottleneck = xception.Xception(weights='imagenet', include_top=False, pooling='avg')

In [68]:
train_x_bf = xception_bottleneck.predict(Xtr, batch_size=32, verbose=1)
valid_x_bf = xception_bottleneck.predict(Xv, batch_size=32, verbose=1)

225/225 [==============================] - 3s     


#### feed to logistic reg - ok

In [55]:
logreg = LogisticRegression(multi_class='multinomial', solver='lbfgs')
logreg.fit(train_x_bf, y_train_encode[train_idx])

valid_probs = logreg.predict_proba(valid_x_bf)
valid_preds = logreg.predict(valid_x_bf)

In [56]:
log_loss(yv, valid_probs)

0.0091840332445442142

In [57]:
accuracy_score(y_train_encode[valid_idx], valid_preds)

1.0

#### feed to dense softmax layer - ok

In [86]:
lm = Sequential([Dense(num_classes, activation='softmax', input_shape=(2048,))])
lm.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [87]:
lm.fit(train_x_bf, ytr, epochs=10, batch_size=32, validation_data=(valid_x_bf, yv))

Train on 916 samples, validate on 225 samples
Epoch 1/10
916/916 [==============================] - 0s - loss: 0.7711 - acc: 0.9061 - val_loss: 0.2773 - val_acc: 0.9867
Epoch 2/10
916/916 [==============================] - 0s - loss: 0.1495 - acc: 0.9880 - val_loss: 0.1218 - val_acc: 0.9822
Epoch 3/10
916/916 [==============================] - 0s - loss: 0.0683 - acc: 0.9891 - val_loss: 0.0730 - val_acc: 0.9867
Epoch 4/10
916/916 [==============================] - 0s - loss: 0.0444 - acc: 0.9902 - val_loss: 0.0617 - val_acc: 0.9822
Epoch 5/10
916/916 [==============================] - 0s - loss: 0.0340 - acc: 0.9902 - val_loss: 0.0531 - val_acc: 0.9867
Epoch 6/10
916/916 [==============================] - 0s - loss: 0.0262 - acc: 0.9924 - val_loss: 0.0545 - val_acc: 0.9822
Epoch 7/10
916/916 [==============================] - 0s - loss: 0.0231 - acc: 0.9913 - val_loss: 0.0509 - val_acc: 0.9822
Epoch 8/10
916/916 [==============================] - 0s - loss: 0.0189 - acc: 0.9924 - val_l

### put preprocess step in a model - xception - dense softmax layer - ok 

In [92]:
x_train = np.empty((len(labels), 299, 299, 3), dtype='float32')
def read_img(img_id, train_or_test, size):
    img = image.load_img(join(data_dir, train_or_test, '%s.jpg' % img_id), target_size=size)
    img = image.img_to_array(img)
    return img

for i, img_id in enumerate(labels['id']):
    img = read_img(img_id, 'train', (299, 299))
    x_train[i] = img

In [ ]:
Xtr = x_train[train_idx]
Xv = x_train[valid_idx]
base_model = xception.Xception(weights='imagenet', include_top=False, pooling='avg')
inputs = Input(shape=(299, 299, 3))
x = Lambda(xception.preprocess_input)(inputs)
outputs = base_model(x)
model = Model(inputs, outputs)

In [94]:
train_x_bf = model.predict(Xtr, batch_size=32, verbose=1)
valid_x_bf = model.predict(Xv, batch_size=32, verbose=1)

224/225 [============================>.] - ETA: 0s

In [95]:
lm = Sequential([Dense(num_classes, activation='softmax', input_shape=(2048,))])
lm.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [96]:
lm.fit(train_x_bf, ytr, epochs=10, batch_size=32, validation_data=(valid_x_bf, yv))

Train on 916 samples, validate on 225 samples
Epoch 1/10
916/916 [==============================] - 1s - loss: 0.7613 - acc: 0.9258 - val_loss: 0.2718 - val_acc: 0.9822
Epoch 2/10
916/916 [==============================] - 0s - loss: 0.1460 - acc: 0.9869 - val_loss: 0.1098 - val_acc: 0.9822
Epoch 3/10
916/916 [==============================] - 0s - loss: 0.0678 - acc: 0.9880 - val_loss: 0.0710 - val_acc: 0.9822
Epoch 4/10
916/916 [==============================] - 0s - loss: 0.0428 - acc: 0.9902 - val_loss: 0.0666 - val_acc: 0.9822
Epoch 5/10
916/916 [==============================] - 0s - loss: 0.0331 - acc: 0.9913 - val_loss: 0.0553 - val_acc: 0.9822
Epoch 6/10
916/916 [==============================] - 0s - loss: 0.0270 - acc: 0.9913 - val_loss: 0.0526 - val_acc: 0.9822
Epoch 7/10
916/916 [==============================] - 0s - loss: 0.0235 - acc: 0.9913 - val_loss: 0.0496 - val_acc: 0.9822
Epoch 8/10
916/916 [==============================] - 0s - loss: 0.0195 - acc: 0.9924 - val_l

### make into one model - no longer works

In [80]:
base_model = xception.Xception(weights='imagenet', include_top=False, pooling='avg')
for layer in base_model.layers:
    layer.trainable = False
inputs = Input(shape=(299, 299, 3))
x = Lambda(xception.preprocess_input)(inputs)
x = base_model(x)
outputs = Dense(num_classes, activation='softmax')(x)
model = Model(inputs, outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [82]:
model.fit(x=Xtr, y=ytr, batch_size=32, epochs=3, verbose=1, validation_data=(Xv, yv))

Train on 916 samples, validate on 225 samples
Epoch 1/3
916/916 [==============================] - 16s - loss: 0.8839 - acc: 0.8570 - val_loss: 2.3357 - val_acc: 0.1378
Epoch 2/3
916/916 [==============================] - 15s - loss: 0.2263 - acc: 0.9793 - val_loss: 2.3743 - val_acc: 0.1111
Epoch 3/3
916/916 [==============================] - 15s - loss: 0.1251 - acc: 0.9803 - val_loss: 2.3750 - val_acc: 0.1556
